# Setup

In [1]:
CASSANDRA_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

CASSANDRA_CLUSTER_NAME = "cassandra-cluster.mavasbel.vpn.itam.mx"
CASSANDRA_TOTAL_NODES = 3

CASSANDRA_NODE_IPS = ["10.15.20.2"] * CASSANDRA_TOTAL_NODES
CASSANDRA_NODE_NAMES = [f"cassandra-node-{i+1}" for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_HOSTNAMES = [
    f"{CASSANDRA_NODE_NAMES[i]}.mavasbel.vpn.itam.mx"
    for i in range(CASSANDRA_TOTAL_NODES)
]
CASSANDRA_NODE_GOSSIP_PORTS = [7000 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_RPC_PORTS = [9040 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_SSL_GOSSIP_PORTS = [7500 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_JMX_PORTS = [7200 + (i + 1) for i in range(0, CASSANDRA_TOTAL_NODES)]

CASSANDRA_CA_CERT_PASSWORD = "cassandra_cluster_ca_cert_passowrd"
CASSANDRA_NODE_CERT_PASSWORD = "cassandra_cluster_cert_passowrd"
CASSANDRA_INIT_USER = "cassandra"
CASSANDRA_INIT_PASSWORD = "cassandra"

CASSANDRA_WORKDIR = "/var/lib/cassandra"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR = os.path.join(LOCALHOST_WORKDIR, "cluster_certs")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)
os.environ.setdefault("CQLENG_ALLOW_SCHEMA_MANAGEMENT", "True")

'True'

# Session creation

In [3]:
from cassandra.cluster import Cluster
from cassandra.connection import DefaultEndPoint
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

cassandra_nodes = [
    DefaultEndPoint(CASSANDRA_NODE_HOSTNAMES[j], CASSANDRA_NODE_RPC_PORTS[j])
    for j in range(CASSANDRA_TOTAL_NODES)
]
print(
    f"🔗 Connecting to: {[f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes]}"
)
print(
    f"JDBC URL: jdbc:cassandra://{','.join([f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes])}"
)

auth_provider = PlainTextAuthProvider(
    username=CASSANDRA_INIT_USER, password=CASSANDRA_INIT_PASSWORD
)
cluster = Cluster(contact_points=cassandra_nodes, auth_provider=auth_provider)

session = cluster.connect()
session.row_factory = dict_factory
print(f"✅ Connected to cluster: {cluster.metadata.cluster_name}")
print(f"🌐 Nodes found: {len(cluster.metadata.all_hosts())}")

🔗 Connecting to: ['cassandra-node-1.mavasbel.vpn.itam.mx:9041', 'cassandra-node-2.mavasbel.vpn.itam.mx:9042', 'cassandra-node-3.mavasbel.vpn.itam.mx:9043']
JDBC URL: jdbc:cassandra://cassandra-node-1.mavasbel.vpn.itam.mx:9041,cassandra-node-2.mavasbel.vpn.itam.mx:9042,cassandra-node-3.mavasbel.vpn.itam.mx:9043
✅ Connected to cluster: cassandra-cluster.mavasbel.vpn.itam.mx
🌐 Nodes found: 3


In [4]:
import pprint
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)
pd.options.display.html.use_mathjax = True
pd.options.display.html.border = 1
pd.options.display.html.table_schema = False
pd.options.display.expand_frame_repr = True

display(
    pd.DataFrame(
        [cassandra_host.__dict__ for cassandra_host in cluster.metadata.all_hosts()]
    )
    .sort_index()
    .transpose()
)

,0,1,2
endpoint,cassandra-node-1.mavasbel.vpn.itam.mx:9041,10.15.20.2:9042,10.15.20.2:9043
conviction_policy,<cassandra.policies.SimpleConvictionPolicy object at 0x0000023772FAF0E0>,<cassandra.policies.SimpleConvictionPolicy object at 0x0000023772FB1480>,<cassandra.policies.SimpleConvictionPolicy object at 0x0000023772FB1810>
host_id,12540fbb-49cb-4d4e-8309-e85ebb42c8a8,18c31b84-d51d-440d-8571-8b702a32505c,78842bb5-ef71-4729-b322-08edaf306acc
_datacenter,dc1,dc1,dc1
_rack,rack1,rack1,rack1
lock,<unlocked _thread.RLock object owner=0 count=0 at 0x0000023770E6A0C0>,<unlocked _thread.RLock object owner=0 count=0 at 0x00000237504B2A80>,<unlocked _thread.RLock object owner=0 count=0 at 0x0000023770E07C80>
is_up,True,True,True
listen_address,172.19.0.2,NaN,NaN
listen_port,7001.0,NaN,NaN
broadcast_address,10.15.20.2,10.15.20.2,10.15.20.2


In [5]:
# from cassandra.cluster import ResultSet

# peers_v2: ResultSet = session.execute("SELECT * FROM system.peers_v2")
# locals: ResultSet = session.execute("SELECT * FROM system.local")

# for peer_v2 in peers_v2:
#     print(" | ".join([f"{col}: {peer_v2[col]}" for col in peers_v2.column_names]))
# print("=" * 100)
# for local in locals:
#     print(" | ".join([f"{col}: {local[col]}" for col in locals.column_names]))
# print("=" * 100)

### Keyspace

In [6]:
keyspace_name = "generic_analytics"
if CASSANDRA_START_FROM_SCRATCH:
    session.execute(f"DROP KEYSPACE IF EXISTS {keyspace_name}")

In [7]:
keyspace_name = "generic_analytics"
session.execute(
    f"""
CREATE KEYSPACE IF NOT EXISTS {keyspace_name} 
WITH replication = {{
    'class': 'NetworkTopologyStrategy', 
    'dc1': {CASSANDRA_TOTAL_NODES}
}}
"""
)
session.set_keyspace(keyspace_name)

# Create Table
# Partition Key: city (distributes data)
# Clustering Column: user_id (sorts data within city)
session.execute(
    """
CREATE TABLE IF NOT EXISTS user_metrics (
    city text,
    user_id uuid,
    username text,
    session_duration int,
    last_access timestamp,
    PRIMARY KEY (city, user_id)
);
"""
)

### Insert

In [8]:
from faker import Faker

fake = Faker()

In [ ]:
import uuid
from datetime import datetime
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel
from cassandra.query import BatchStatement, BatchType

# 1. Prepare your statement outside the loop
query = """
INSERT INTO user_metrics (city, user_id, username, session_duration, last_access)
VALUES (?, ?, ?, ?, ?)
"""
prepared = session.prepare(query)
# prepared.consistency_level = ConsistencyLevel.QUORUM
# statement = SimpleStatement(query, consistency_level=ConsistencyLevel.QUORUM)

# 2. Create the Batch object
# LOGGED ensures atomicity but adds disk overhead, UNLOGGED is faster
batch_records = 200
batch = BatchStatement(batch_type=BatchType.LOGGED)
batch.consistency_level = ConsistencyLevel.QUORUM
print(f"Preparing batch of {batch_records} records...")
for _ in range(batch_records):
    batch.add(
        prepared,
        (
            fake.city(),
            uuid.uuid4(),
            fake.user_name(),
            fake.random_int(min=1, max=3600),
            datetime.now(),
        ),
    )

# 3. Execute the entire batch at once
session.execute(batch)
print("Batch successfully committed to the cluster.")

Preparing batch of 200 records...
Batch successfully committed to the cluster.


### Query

In [17]:
from cassandra.cluster import ResultSet
from typing import cast

print(cast(ResultSet, session.execute("SELECT count(*) FROM user_metrics")).one())

{'count': 400}


In [18]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT * FROM user_metrics LIMIT 100")
for row in rows.current_rows:
    print(row)

{'city': 'Wernerport', 'user_id': UUID('730d3dc3-c650-420a-8c84-284d73858dfd'), 'last_access': datetime.datetime(2026, 1, 20, 21, 28, 50, 217000), 'session_duration': 2499, 'username': 'erivera'}
{'city': 'East Jose', 'user_id': UUID('37a7bc1e-3bee-4e4c-a5fb-19bf9c582829'), 'last_access': datetime.datetime(2026, 1, 20, 21, 34, 54, 804000), 'session_duration': 2043, 'username': 'rogersjesse'}
{'city': 'North Tyler', 'user_id': UUID('8d0ae574-bb0f-40d1-b633-bac1ae0f60b4'), 'last_access': datetime.datetime(2026, 1, 20, 21, 28, 50, 226000), 'session_duration': 1175, 'username': 'morenobelinda'}
{'city': 'North Tyler', 'user_id': UUID('b377b707-5c85-40bf-8c79-014e0899746e'), 'last_access': datetime.datetime(2026, 1, 20, 21, 28, 50, 209000), 'session_duration': 3208, 'username': 'christopherhayes'}
{'city': 'Fowlerburgh', 'user_id': UUID('addae533-8d44-4f68-a4dd-0ddaf94aeab5'), 'last_access': datetime.datetime(2026, 1, 20, 21, 28, 50, 238000), 'session_duration': 1727, 'username': 'pcochran'

In [19]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT DISTINCT city FROM user_metrics", [])
for row in rows.current_rows:
    print(row)

{'city': 'Wernerport'}
{'city': 'East Jose'}
{'city': 'North Tyler'}
{'city': 'Fowlerburgh'}
{'city': 'South Desiree'}
{'city': 'West Katrinaland'}
{'city': 'Williamside'}
{'city': 'North Monica'}
{'city': 'Douglasburgh'}
{'city': 'East Nathan'}
{'city': 'East Timothy'}
{'city': 'Port Jessica'}
{'city': 'Davisland'}
{'city': 'West Joshua'}
{'city': 'Port Patrickborough'}
{'city': 'Durhamville'}
{'city': 'Christopherside'}
{'city': 'East Anthony'}
{'city': 'East Cassandra'}
{'city': 'Kimberlyville'}
{'city': 'North Charleshaven'}
{'city': 'Port Jacob'}
{'city': 'Amberborough'}
{'city': 'Shawnton'}
{'city': 'East Lori'}
{'city': 'Lake Vickishire'}
{'city': 'Port William'}
{'city': 'West Cody'}
{'city': 'Haleyland'}
{'city': 'Priceborough'}
{'city': 'Josephstad'}
{'city': 'Port Christopherville'}
{'city': 'North Sandratown'}
{'city': 'East Jasminestad'}
{'city': 'Jermainemouth'}
{'city': 'Port Randall'}
{'city': 'Weeksfurt'}
{'city': 'Lake Stevenberg'}
{'city': 'Huntville'}
{'city': 'West

### Find nodes storing random data

In [23]:
import random
import pprint

random_token = random.randint(-9223372036854775808, 9223372036854775807)
query = "SELECT * FROM user_metrics WHERE token(city) >= %s LIMIT 1"
row: dict = cast(ResultSet, session.execute(query, [random_token])).one()
print(f"Random user_metric: {pprint.pformat(row)}")

prepared = session.prepare("SELECT * FROM user_metrics WHERE city=? AND user_id=?")
bound = prepared.bind([row['city'], row['user_id']])
routing_key = bound.routing_key
nodes = cluster.metadata.get_replicas(keyspace_name, routing_key)

print(f"Nodes storing '{row['city']}':")
for node in nodes:
    print(f" - Host: {node.address}, Gossip Port: {node.broadcast_port}")

Random user_metric: {'city': 'Christensenton',
 'last_access': datetime.datetime(2026, 1, 20, 20, 15, 8, 646000),
 'session_duration': 3594,
 'user_id': UUID('924c7cbd-db9a-4bc6-b812-9d4cc6d1e04b'),
 'username': 'thomasterry'}
Nodes storing 'Christensenton':
 - Host: 10.15.20.2, Gossip Port: 7002
 - Host: cassandra-node-1.mavasbel.vpn.itam.mx, Gossip Port: 7001
 - Host: 10.15.20.2, Gossip Port: 7003


### ORM-like

In [24]:
from typing import cast
from cassandra.cqlengine import columns
from cassandra.cqlengine.query import ModelQuerySet
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table, create_keyspace_network_topology
from cassandra.cqlengine import connection

# 1. Connect the engine
connection.set_session(session)


# 2. Define your "Generic" Model
class UserMetrics(Model):
    __table_name__ = "user_metrics"

    # FIRST primary_key=True is the Partition Key
    # city = columns.Text(primary_key=True)
    city = columns.Text(primary_key=True, partition_key=True)

    # SECOND primary_key=True is the Clustering Key
    user_id = columns.UUID(primary_key=True, default=uuid.uuid4)

    # Attributes (Data)
    username = columns.Text(index=True)
    session_duration = columns.Integer()
    last_access = columns.DateTime()


# 3. Create Keyspace and Sync Table (Equivalent to CREATE TABLE)
create_keyspace_network_topology(keyspace_name, {"dc1": CASSANDRA_TOTAL_NODES})
sync_table(UserMetrics)

In [25]:
# 4. Use it like an ORM
# Create UserMetrics
new_metric: UserMetrics = UserMetrics.create(
    city=fake.city(),
    username=fake.name(),
    session_duration=120,
    last_access=datetime.now(),
)
new_metric.save()
print(f"Saved user_metris: {new_metric}")


# 5. Query UserMetrics
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    print(f"Query user_metris: {user_metric}")

Saved user_metris: UserMetrics <city=Port Sierra, user_id=4f61c310-4f1b-46c3-ba2d-0e717e05e6da>
Query user_metris: UserMetrics <city=South Dennisfurt, user_id=7d399476-6f05-443d-8cca-5cb8882c3583>
Query user_metris: UserMetrics <city=Snyderborough, user_id=2ccbbbde-0536-4629-8a7d-603a3ca5aefa>
Query user_metris: UserMetrics <city=Richardville, user_id=df99a6be-eae9-481c-bd6a-9ef7b7accd38>
Query user_metris: UserMetrics <city=Port Rebecca, user_id=426d25d0-659b-4113-9e88-94589acff342>
Query user_metris: UserMetrics <city=Lake Heidi, user_id=b4b46e23-34ab-40ad-9ee9-4e5399cea448>
Query user_metris: UserMetrics <city=New Katherinemouth, user_id=100f80b9-600a-463e-a68b-cf9a9f8bcaa9>
Query user_metris: UserMetrics <city=Lake Thomasport, user_id=3d71d1d5-3874-4e25-992b-41b59ffee639>
Query user_metris: UserMetrics <city=Boydton, user_id=43154408-9e1b-4dc1-976a-e5414a8b6a6b>
Query user_metris: UserMetrics <city=New Kathleen, user_id=993754e9-a68c-414f-a1cf-f97a733765e1>
Query user_metris: UserM

### Insert/Create

In [26]:
from typing import cast

cast(ModelQuerySet, UserMetrics.ttl(86400)).create(
    city=fake.city(),
    username=fake.user_name(),
    session_duration=fake.random_int(10, 1000),
    last_access=fake.date_time(),
)

UserMetrics(city='Martinezshire', user_id=UUID('f0e25427-cc3c-45a4-af18-02957c7496bc'), username='jordanross', session_duration=558, last_access=datetime.datetime(1980, 2, 5, 0, 54, 56))

In [27]:
from typing import cast
from cassandra.cqlengine.query import BatchQuery
from cassandra.cqlengine.query import ModelQuerySet

with BatchQuery() as b:
    for _ in range(100):
        cast(ModelQuerySet, UserMetrics.batch(b)).create(
            city=fake.city(),
            username=fake.user_name(),
            session_duration=fake.random_int(10, 1000),
            last_access=fake.date_time(),
        )

### Delete

In [28]:
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    user_metric.delete()
    print(f"Delete user_metris: {user_metric}")

Delete user_metris: UserMetrics <city=South Dennisfurt, user_id=7d399476-6f05-443d-8cca-5cb8882c3583>
Delete user_metris: UserMetrics <city=Snyderborough, user_id=2ccbbbde-0536-4629-8a7d-603a3ca5aefa>
Delete user_metris: UserMetrics <city=Richardville, user_id=df99a6be-eae9-481c-bd6a-9ef7b7accd38>
Delete user_metris: UserMetrics <city=East Jamesshire, user_id=acc65ce9-7bdd-488f-b213-1d5dc5118e72>
Delete user_metris: UserMetrics <city=Port Rebecca, user_id=426d25d0-659b-4113-9e88-94589acff342>
Delete user_metris: UserMetrics <city=Lake Heidi, user_id=b4b46e23-34ab-40ad-9ee9-4e5399cea448>
Delete user_metris: UserMetrics <city=New Katherinemouth, user_id=100f80b9-600a-463e-a68b-cf9a9f8bcaa9>
Delete user_metris: UserMetrics <city=Rebeccaborough, user_id=664fc5b8-5fdb-4c16-bdaf-a9775aec613e>
Delete user_metris: UserMetrics <city=Lake Thomasport, user_id=3d71d1d5-3874-4e25-992b-41b59ffee639>
Delete user_metris: UserMetrics <city=Boydton, user_id=43154408-9e1b-4dc1-976a-e5414a8b6a6b>
Delete u

In [29]:
# from cassandra.cqlengine import connection

# # This isn't a compaction, but it forces RAM data to disk so the background compaction process can see it.
# connection.execute(f"ALTER TABLE {keyspace_name}.user_metrics WITH gc_grace_seconds = 10")